# News Summarization

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [3]:
from datasets import load_dataset

# BBC News Summary
dataset = load_dataset("SurAyush/News_Summary_Dataset")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 2224
    })
})

In [5]:
split_dataset = dataset['train'].train_test_split(test_size=0.1)

In [6]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 2001
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 223
    })
})

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = 'google-t5/t5-small'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [8]:
inputs = tokenizer("Hello, I need to summarized by the model")

inputs

{'input_ids': [8774, 6, 27, 174, 12, 21603, 26, 57, 8, 825, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁Hello',
 ',',
 '▁I',
 '▁need',
 '▁to',
 '▁summarize',
 'd',
 '▁by',
 '▁the',
 '▁model',
 '</s>']

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
# model size
model.num_parameters()/1_000_000

60.506624

In [12]:
tokenizer.model_max_length

512

In [13]:
max_input_length = 512
max_target_length = 256

def preprocess_function(examples):

    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )

    # unlike translation of languages, here we have the same tokenizer for labels
    labels = tokenizer(
        examples["summary"],
        max_length=max_target_length,
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]            # we only need input_ids of lables

    return model_inputs

In [14]:
tokenized_dataset = split_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

In [15]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2001
    })
    test: Dataset({
        features: ['article', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 223
    })
})

In [16]:
!pip install rouge_score
!pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=06ef47dd6ed3747474ed3ac84b5dfc89fb021862a975096f0ec8e503df627f48
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


In [17]:
import evaluate

rouge_score = evaluate.load("rouge")

In [18]:
generated_summary = "rise in oil prices have created riots"
reference_summary = "riots caused by rise in oil prices"

scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': np.float64(0.7142857142857143),
 'rouge2': np.float64(0.5),
 'rougeL': np.float64(0.5714285714285714),
 'rougeLsum': np.float64(0.5714285714285714)}

The rouge1 variant is the overlap of unigrams — this is just a fancy way of saying the overlap of words and is exactly the metric we’ve discussed above.

rouge2 measures the overlap between bigrams (think the overlap of pairs of words).

rougeL and rougeLsum measure the longest matching sequences of words by looking for the longest common substrings in the generated and reference summaries. The “sum” in rougeLsum refers to the fact that this metric is computed over a whole summary, while rougeL is computed as the average over individual sentences.

## Let's fine tune the T5 small model

In [19]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8

logging_steps = len(tokenized_dataset["train"]) // batch_size

model_name = "news-summarizer-t5"

args = Seq2SeqTrainingArguments(
    output_dir= model_name,
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,                   # limiting the number of saves
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,        # seq2seq evaluation
    logging_steps=logging_steps,
    push_to_hub=True,
    fp16 = True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
!pip install nltk

In [21]:
import nltk

nltk.download("punkt")       # punctuation rules
nltk.download("punkt_tab")       # punctuation rules

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [22]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [23]:
import numpy as np
from nltk.tokenize import sent_tokenize

def compute_metrics(eval_pred):
    '''
        similar to translation compute_metrics
    '''

    predictions, labels = eval_pred


    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 with <PAD>
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE score
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )

    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [24]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [25]:
tokenized_dataset = tokenized_dataset.remove_columns(
    split_dataset["train"].column_names
)

In [26]:
features = [tokenized_dataset["train"][i] for i in range(2)]
collated_samples = data_collator(features)

In [27]:
collated_samples.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [28]:
from huggingface_hub import notebook_login, whoami

notebook_login()

In [29]:
whoami()

{'type': 'user',
 'id': '66a797b589b3e71262932d0d',
 'name': 'SurAyush',
 'fullname': 'Ayush Sur',
 'email': 'ayushsur26@gmail.com',
 'emailVerified': True,
 'canPay': False,
 'periodEnd': None,
 'isPro': False,
 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/noauth/RZJZW_w0wdVoOmQY250lR.png',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'Google Colab',
   'role': 'write',
   'createdAt': '2025-03-02T10:56:32.713Z'}}}

In [30]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-30-388ad16f7736>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [31]:
# before fine-tuning
trainer.evaluate()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ayushsur26 (ayushsur26-national-institute-of-technology-kurukshetra) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 5.750208377838135,
 'eval_model_preparation_time': 0.0049,
 'eval_rouge1': 13.918,
 'eval_rouge2': 8.1441,
 'eval_rougeL': 11.6651,
 'eval_rougeLsum': 12.5001,
 'eval_runtime': 14.2089,
 'eval_samples_per_second': 15.694,
 'eval_steps_per_second': 1.971}

 - 'eval_rouge1': 13.918,
 - 'eval_rouge2': 8.1441,
 - 'eval_rougeL': 11.6651,
 - 'eval_rougeLsum': 12.5001

In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Rouge1,Rouge2,Rougel,Rougelsum
1,0.868900,0.658052,0.004900,18.874500,16.231400,18.199100,18.328700
2,0.662900,0.638466,0.004900,19.370500,17.127700,18.868500,18.959400
3,0.611400,0.629368,0.004900,19.395100,17.211300,18.931500,18.984800
4,0.571000,0.619669,0.004900,19.868400,17.823400,19.464600,19.540100
5,0.545100,0.619271,0.004900,19.898100,17.985100,19.508300,19.517700
6,0.519400,0.620252,0.004900,19.867500,17.952100,19.543400,19.604600
7,0.489400,0.616615,0.004900,19.862200,17.961600,19.479100,19.566900
8,0.487200,0.617713,0.004900,19.884900,17.993900,19.532800,19.591800


TrainOutput(global_step=2008, training_loss=0.5943762056856041, metrics={'train_runtime': 652.6637, 'train_samples_per_second': 24.527, 'train_steps_per_second': 3.077, 'total_flos': 2164271365816320.0, 'train_loss': 0.5943762056856041, 'epoch': 8.0})

In [34]:
trainer.push_to_hub(commit_message='Training Complete', tags="summarization")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

events.out.tfevents.1743435457.8e470de03c02.749.0:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SurAyush/news-summarizer-t5/commit/74a4bd4afd727d29502f799a9e5b0315a0d9021b', commit_message='Training Complete', commit_description='', oid='74a4bd4afd727d29502f799a9e5b0315a0d9021b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SurAyush/news-summarizer-t5', endpoint='https://huggingface.co', repo_type='model', repo_id='SurAyush/news-summarizer-t5'), pr_revision=None, pr_num=None)

### Using the model from pipeline

In [35]:
from transformers import pipeline

hub_model_id = 'SurAyush/news-summarizer-t5'
summarizer = pipeline('summarization',model = hub_model_id)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Device set to use cuda:0


In [36]:
text = '''
The death toll from a major earthquake in Myanmar has risen to 2,056, the junta said on Monday. More than 3,900 people were injured. A statement from a junta spokesperson said that 270 more people were still missing. It has also declared a week of national mourning and directed that national flags will fly at half-mast until April 6 “in sympathy for the loss of life and damages.” The announcement came as the tempo and urgency of rescue efforts wound down in Mandalay, one of the worst-affected cities and the country's second-largest, with more than 1.7 million inhabitants.

"The situation is so dire that it's hard to express what is happening," said Aung Myint Hussein, chief administrator of Mandalay's Sajja North mosque, AFP reported.
According to AFP, people camped out in the streets across Mandalay for a third successive night because they were either unable to return to their ruined homes or were nervous about the repeated aftershocks that rattled the city over the weekend.
Some had tents, but many, including young children, simply slept on blankets in the middle of the roads, trying to stay as far away from buildings.
Among those killed in the quake were also three Chinese nationals and two French people, reported AFP, citing China's state media and the foreign ministry in Paris.

However, with communications down in much of Myanmar, the true scale of the disaster has yet to emerge and the death toll is expected to rise significantly.
The 7.7 earthquake left wide cracks on roads, brought down buildings, and sent tremors across neighbouring countries, including China, Thailand, Vietnam and parts of India.
'''

output = summarizer(text)

In [40]:
summary = output[0]['summary_text']

In [46]:
import re

def capitalize_sentences(text):
    sentences = re.split(r'([.!?]\s*)', text)  # Split text while keeping punctuation
    capitalized_sentences = [s.capitalize() for s in sentences]
    result = ''.join(capitalized_sentences)

    result = re.sub(r'\bindia\b', 'India', result, flags=re.IGNORECASE)

    return result

In [48]:
capitalize_sentences(summary)

"The death toll from a major earthquake in myanmar has risen to 2,056, the junta said on monday.The 7.7 earthquake left wide cracks on roads, brought down buildings, and sent tremors across neighbouring countries, including china, thailand, vietnam and parts of India.The announcement came as the tempo and urgency of rescue efforts wound down in mandalay, one of the worst-affected cities and the country's second-largest, with more than 1.7 million inhabitants."